In [21]:
%matplotlib inline
import matplotlib.pyplot as pp
import numpy as np
from scipy import signal
import librosa
import librosa.display
from IPython import display
from copy import deepcopy

# the pra library
import pyroomacoustics as pra

In [41]:
# A simple wrapper class for (1-channel) audio data
# data is a 1-D NumPy array containing the data
# rate is a number expressing the samples per second
class Audio:
    def __init__(self, data, rate):
        self.data = data
        self.rate = rate
    def play(self):
        return display.Audio(self.data, rate=self.rate)
    def plot_wave(self):
        librosa.display.waveplot(self.data, sr=self.rate)
    def plot_spectrum(self):
        n_fft = int(self.rate / 20)
        D = librosa.amplitude_to_db(np.abs(librosa.stft(self.data, n_fft)), ref=np.max)
        librosa.display.specshow(D, y_axis='linear', sr=self.rate, hop_length=n_fft/4)
    @classmethod
    def fromfile(cls, fn):
        return cls(*librosa.load(fn, sr=None))

## Basic Example

In [101]:
'''Room Parameters and Room Creation'''
rt60 = 0.5 # the time it takes for the room impulse reponse (RIR) to decay by 60dB
room_dim = [10, 10, 10] # meters

# Room is simulated with the "Image Source Model". ISM assumes perfect reflections.
# invert the Sabine's formula to obtain the parameters for the ISM model
e_absorption, max_order = pra.inverse_sabine(rt60, room_dim)
# print(e_absorption, max_order)

# create the room
# fs: sampling rate, max_order: max number of reflections allowed in ISM
room = pra.ShoeBox(room_dim, fs=48000, materials=pra.Material(e_absorption), max_order=max_order)

'''Sound Source Creation'''
# base sound source
sweep = Audio.fromfile("sweep.wav")

# rising long sweep
long_sweep = np.asarray(list(sweep.data) * 5) # sweep * 5
# decreasing long sweep
reverse_sweep = np.asarray(list(sweep.data[::-1]) * 5) # reversed sweep * 5

# room.add_source([7, 1, 3], signal=reverse_sweep, delay=1)
room.add_source([7, 5, 5], signal=long_sweep, delay=1)
# room.add_source([7, 9, 8], signal=reverse_sweep, delay=1)

'''Mic Array Creation'''
mic_locs = np.c_[
    [1, 4.9, 5],
    [1, 5, 5],
    [1, 5.1, 5]
]
# place the array in the room
room.add_microphone_array(mic_locs)
# room.add_microphone([1, 5, 5]) # single microphone

'''Simulation'''
room.simulate()
room.mic_array.to_wav(
    "./simu_recv_sound/basic3mic3src.wav",
    norm=True,
    bitdepth=np.int16
)
result = Audio.fromfile("./simu_recv_sound/basic3mic3src.wav")

### Result Inspection

In [102]:
Audio(long_sweep, sweep.rate).play()

In [103]:
result.play()

## Beamforming

In [110]:
'''Room Parameters and Room Creation'''
room_dim = [10, 10, 10] # meters
m = pra.Material(energy_absorption="panel_fabric_covered_6pcf")
room = pra.ShoeBox(room_dim, fs=48000, materials=m, max_order=0) #simulate perfect situation: no reflections

'''Sound Source Creation'''
# base sound source
sweep = Audio.fromfile("sweep.wav")

# rising long sweep
long_sweep = np.asarray(list(sweep.data) * 5) # sweep * 5
# decreasing long sweep
reverse_sweep = np.asarray(list(sweep.data[::-1]) * 5) # reversed sweep * 5

room.add_source([7, 1, 3], signal=reverse_sweep, delay=1)
room.add_source([7, 5, 5], signal=long_sweep, delay=1)
# room.add_source([7, 9, 8], signal=reverse_sweep, delay=1)

'''Mic Array Creation'''
mic_locs = np.c_[
    [1, 4.9, 5],
    [1, 5, 5],
    [1, 5.1, 5]
]
# place the array in the room
room.add_microphone_array(mic_locs)
# room.add_microphone([1, 5, 5]) # single microphone

'''Simulation'''
room.simulate()
room.mic_array.to_wav(
    "./simu_recv_sound/none_reflection_3mic3src.wav",
    norm=True,
    bitdepth=np.int16
)
result = Audio.fromfile("./simu_recv_sound/none_reflection_3mic3src.wav")

### Result Inspection

In [111]:
Audio(long_sweep, sweep.rate).play()

In [112]:
result.play()